### $\widehat{Z}_a$ Code

In [1]:
import math
from itertools import product
from math import floor, ceil
from copy import copy
from fractions import Fraction
import numpy as np
import scipy

def extract_degree_sequence(M):
    """
        Computes the degrees of the nodes of the plumbing graph given a plumbing matrix
    """
    # D: This uses python functions, numpy would be quicker
    s = M.shape[0]
    degree_sequence = []
    for i in range(s):
        degree_v_i = 0
        for j in range(s):
            if(i!=j):
                degree_v_i += M[i][j]
        degree_sequence.append(degree_v_i)
    return degree_sequence

def extract_weight_vector(M):
    """
        Computes the weights of the nodes of a plumbing graph given the 
    """
    # D: This uses python functions, numpy would be quicker
    s = M.shape[0]
    weight_sequence = []
    for i in range(s):
        weight_sequence.append(M[i, i])
    return weight_sequence

def char_square(M, k):
    s = M.shape[0]
    M_inv = np.linalg.inv(M) # D: the inverse of M is computed every time the function is called
    k = np.array(k)
    k_T = np.transpose(k)
    h = np.matmul(M_inv, k)
    square = np.matmul(k_T, h)
    return square

def zhat(M, s_rep, n):
    """
        Compute the Z-hat invariant of the manifold associated to the plumbing matrix M up to order n
        INPUTS:
            M - Plumbing matrix
            s_rep - spin-c structure representative (must be % 2 with degree vector ??)
            n - order of z-hat invariant
    """
    s = M.shape[0]
    s_rep = np.array(s_rep)
    degree_vector = extract_degree_sequence(M)
    weight_vector = extract_weight_vector(M)
    for i in range(s):
        if (float(s_rep[i])-float(degree_vector[i])) % 2 != 0:
            raise Exception("Your selected spin^c convention is to"
                            " use vectors congruent mod 2 to the"
                            " degree vector, but second parameter"
                            " does not satisfy this.")
    # D: k is the sum of spin_c structure, weight vector and degree vector
    k = s_rep + weight_vector + degree_vector 
    #print(k)
    # D: a is just a copy of s_rep
    a = s_rep
    #k_squared = self.char_vector_properties(k.T)[2] 
    k_squared = char_square(M, np.transpose(k))
    #print(k_squared)
    normalization_term = -(k_squared + 3*s
                            + sum(weight_vector))/4\
                            + sum(k)/2\
                            - sum(weight_vector + degree_vector)/4
    M_inv = np.linalg.inv(M)
    #min_vector = -(1/2)*M_inv*a
    a_T = np.transpose(a)
    # D: a_quadratic_form is the same as char_square(M,a)
    a_quadratic_form = np.matmul(np.matmul(a_T, M_inv), a)
    min_level = (a_quadratic_form)/4
    

    bounding_box = []
    for i in range(0, s): # D: it's safer to cycle through elements of a list (for w in weight_vector: ...)
        if min_level % 1 == 0:
            x = 2*math.sqrt(abs(weight_vector[i])*(n-1))
        else:
            x = 2*math.sqrt(abs(weight_vector[i])*n)
        bounding_box.append([-x, x])

    # D: F_supp is a list of lists, each list contains the possible values for the i-th coordinate
    F_supp = []
    for i in range(s):
        if degree_vector[i] == 0:
            if bounding_box[i][0] <= -2 and bounding_box[i][1] >= 2: # I
                F_supp.append([-2, 0, 2])
            elif bounding_box[i][0] <= -2 and bounding_box[i][1] < 2: # II
                F_supp.append([-2, 0])
            elif bounding_box[i][0] > -2 and bounding_box[i][1] >= 2: # III
                F_supp.append([0, 2])
            else:
                F_supp.append([0])
        elif degree_vector[i] == 1:
            if bounding_box[i][0] <= -1 and bounding_box[i][1] >= 1: # I
                F_supp.append([-1, 1])
            elif bounding_box[i][0] <= -1 and bounding_box[i][1] < 1: # II
                F_supp.append([-1])
            elif bounding_box[i][0] > -1 and bounding_box[i][1] >= 1: # III
                F_supp.append([1])
            else:
                return 0
        elif degree_vector[i] == 2:
            F_supp.append([0])
        else:
            r = degree_vector[i]-2
            values = []
            if bounding_box[i][0] <= -r: 
                values.append(-r)
                for j in range(1, floor((-r-bounding_box[i][0])/2)+1):
                    values.append(-r - 2*j)
            if bounding_box[i][1] >=r:
                values.append(r)
                for j in range(1, floor((bounding_box[i][1]-r)/2)+1):
                    values.append(r + 2*j)
            if len(values)==0:
                return 0
            # D: why not just use values instead of copy(values)?
            F_supp.append(copy(values))
    iterator = product(*F_supp)

    def F_hat_pre_comp(y):
        F = 1
        for i in range(0, s):
            if degree_vector[i] == 0:
                if y[i] == 0:
                    F = -2*F
                elif y[i] != 2 and y[i]!= -2: # D: This can never happen, y is 0 +- 2
                    F = 0
                    return F
            elif degree_vector[i] == 1:
                if y[i] == 1:
                    F = -F
                elif y[i] != -1: # D: This can never happen, y is +- 1
                    F = 0
                    return F
            elif degree_vector[i] == 2:
                if y[i] != 0: # D: This can never happen, y is 0
                    F = 0
                    return F
            else:
                if abs(y[i]) >= degree_vector[i]-2:
                    F = F*Fraction(1, 2)*(np.sign(y[i])**degree_vector[i])
                    F = F*math.comb(int(Fraction(degree_vector[i]
                                    + abs(y[i]), 2))-2, degree_vector[i]-3)
                else: # D: This too can never happen based on how F_supp is defined
                    F = 0
                    return F
        return F
    
    # n is the number of terms in the z-hat invariant
    exponents = [ceil(min_level) + i for i in range(n)]
    coefficients = np.zeros((n,))


    i = 0
    for y in iterator:
        i += 1
        #y = Matrix(y).T
        y = np.array(y)
        #c = -((y.T*M_inv*y)[0,0])/4
        y_T = np.transpose(y)
        c = -1*(np.matmul(np.matmul(y_T, M_inv), y))/4
        #if frac(min_level) == 0:
        if (min_level % 1 ==0): # D: you can avoid using this if statement
            if c <= n-1:
                #x = (1/2)*M_inv*(y-a)
                x = Fraction(1,2)*np.matmul(M_inv, y-a)
                # now accounting for arithmetic floating errors
                x= np.around(x.astype(np.float64), 4)
                #if x gin MatrixSpace(ZZ, s, 1):
                if np.all(np.mod(x, 1) == 0.0):
                    ind = c
                    coefficients[ind] = coefficients[ind] + F_hat_pre_comp(y)
        else:
            if c <= n:
                #x = (1/2)*M_inv*(y-a)
                x = Fraction(1,2)*np.matmul(M_inv, y-a)
                # now accounting for arithmetic floating errors
                x= np.around(x.astype(np.float64), 4)
                #if x in MatrixSpace(ZZ, s, 1):
                if np.all(np.mod(x, 1) == 0.0):
                    ind = floor(c)
                    coefficients[ind] = coefficients[ind] + F_hat_pre_comp(y)

    zhat_list = [(coefficients[i], exponents[i] + normalization_term) for i in range(0, n)]

    return zhat_list

In [100]:
# Set up data
s = M.shape[0]
s_rep = np.array(s_rep)
degree_vector = extract_degree_sequence(M)
weight_vector = extract_weight_vector(M)


# Assert that the spin-c structure is congruent to the degree vector mod 2
for i in range(s):
    if (float(s_rep[i])-float(degree_vector[i])) % 2 != 0:
        raise Exception("Your selected spin^c convention is to"
                        " use vectors congruent mod 2 to the"
                        " degree vector, but second parameter"
                        " does not satisfy this.")


# Compute the min level
a = s_rep
M_inv = np.linalg.inv(M)
a_T = np.transpose(a)
a_quadratic_form = np.matmul(np.matmul(a_T, M_inv), a)
min_level = (a_quadratic_form)/4
   
# Compute the normalization term
k = s_rep + weight_vector + degree_vector 
k_squared = char_square(M, np.transpose(k))
normalization_term = -(k_squared + 3*s
                        + sum(weight_vector))/4\
                        + sum(k)/2\
                        - sum(weight_vector + degree_vector)/4


# Establish bounding box
bounding_box = []
for i in range(0, s):
    if min_level % 1 == 0:
        x = 2*math.sqrt(abs(weight_vector[i])*(n-1))
    else:
        x = 2*math.sqrt(abs(weight_vector[i])*n)
    bounding_box.append([-x, x])

# Build F_supp
F_supp = []
for i in range(s):
    if degree_vector[i] == 0:
        if bounding_box[i][0] <= -2 and bounding_box[i][1] >= 2: # I
            F_supp.append([-2, 0, 2])
        elif bounding_box[i][0] <= -2 and bounding_box[i][1] < 2: # II
            F_supp.append([-2, 0])
        elif bounding_box[i][0] > -2 and bounding_box[i][1] >= 2: # III
            F_supp.append([0, 2])
        else:
            F_supp.append([0])
    elif degree_vector[i] == 1:
        if bounding_box[i][0] <= -1 and bounding_box[i][1] >= 1: # I
            F_supp.append([-1, 1])
        elif bounding_box[i][0] <= -1 and bounding_box[i][1] < 1: # II
            F_supp.append([-1])
        elif bounding_box[i][0] > -1 and bounding_box[i][1] >= 1: # III
            F_supp.append([1])
        else:
            1+1
            #return 0
    elif degree_vector[i] == 2:
        F_supp.append([0])
    else:
        r = degree_vector[i]-2
        values = []
        if bounding_box[i][0] <= -r: 
            values.append(-r)
            for j in range(1, floor((-r-bounding_box[i][0])/2)+1):
                values.append(-r - 2*j)
        if bounding_box[i][1] >=r:
            values.append(r)
            for j in range(1, floor((bounding_box[i][1]-r)/2)+1):
                values.append(r + 2*j)
        if len(values)==0:
            1+1
            #return 0
        # D: why not just use values instead of copy(values)?
        F_supp.append(copy(values))

# Take products of F_supp
degree_vector = np.array(degree_vector)
M_inv = np.linalg.inv(M)
arrays = [ np.array(lst) for lst in F_supp]
grid = np.meshgrid(*arrays)
y_arr = np.array([g.ravel() for g in grid]).T

### Example

In [2]:
M = np.array([[-3, 1, 0, 0, 1, 0, 0], 
              [1, -4, 1, 1, 0, 0, 0], 
              [0, 1, -1, 0, 0, 0, 0], 
              [0, 1, 0, -4, 0, 0, 0], 
              [1, 0, 0, 0, -1, 1, 1], 
              [0, 0, 0, 0, 1, -4, 0], 
              [0, 0, 0, 0, 1, 0, -3]])
degree_sequence = extract_degree_sequence(M)
z_hat_list = zhat(M, degree_sequence, 30000)
for tuple in z_hat_list:
    if(tuple[0] != 0):
        print('{c}q^{e}'.format(c=tuple[0], e=tuple[1]))

-0.5q^1.5769230769224976
-0.5q^7.576923076922498
0.5q^8.576923076922498
0.5q^323.5769230769225
0.5q^381.5769230769225
-0.5q^388.5769230769225
0.5q^556.5769230769225
0.5q^721.5769230769225
-0.5q^1632.5769230769224
-0.5q^1907.5769230769224
0.5q^2261.5769230769224
-0.5q^2278.5769230769224
-0.5q^2428.5769230769224
-0.5q^4143.576923076923
0.5q^4166.576923076923
0.5q^4368.576923076923
-0.5q^4682.576923076923
-0.5q^4896.576923076923
0.5q^4921.576923076923
0.5q^7266.576923076923
0.5q^7532.576923076923
-0.5q^7563.576923076923
0.5q^8253.576923076922
0.5q^8858.576923076922
-0.5q^11591.576923076922
-0.5q^12306.576923076922
0.5q^13182.576923076922
-0.5q^13223.576923076922
-0.5q^13581.576923076922
-0.5q^17326.576923076922
0.5q^17373.576923076922
0.5q^17783.576923076922
-0.5q^18411.576923076922
-0.5q^18833.576923076922
0.5q^18882.576923076922
0.5q^23257.576923076922
0.5q^23731.576923076922
-0.5q^23786.576923076922
0.5q^24998.576923076922
0.5q^26043.576923076922


In [5]:
from plum_python_vectorized import *
M = np.array([[-3, 1, 0, 0, 1, 0, 0], 
              [1, -4, 1, 1, 0, 0, 0], 
              [0, 1, -1, 0, 0, 0, 0], 
              [0, 1, 0, -4, 0, 0, 0], 
              [1, 0, 0, 0, -1, 1, 1], 
              [0, 0, 0, 0, 1, -4, 0], 
              [0, 0, 0, 0, 1, 0, -3]])
degree_sequence = extract_degree_sequence(M)
z_hat_list_fromvec = zhat_vec(M, degree_sequence, 30000)
for tuple in z_hat_list_fromvec:
    if(tuple[0] != 0):
        print('{c}q^{e}'.format(c=tuple[0], e=tuple[1]))

-0.5q^1.5769230769224976
-0.5q^7.576923076922498
0.5q^8.576923076922498
0.5q^323.5769230769225
0.5q^381.5769230769225
-0.5q^388.5769230769225
0.5q^556.5769230769225
0.5q^721.5769230769225
-0.5q^1632.5769230769224
-0.5q^1907.5769230769224
0.5q^2261.5769230769224
-0.5q^2278.5769230769224
-0.5q^2428.5769230769224
-0.5q^4143.576923076923
0.5q^4166.576923076923
0.5q^4368.576923076923
-0.5q^4682.576923076923
-0.5q^4896.576923076923
0.5q^4921.576923076923
0.5q^7266.576923076923
0.5q^7532.576923076923
-0.5q^7563.576923076923
0.5q^8253.576923076922
0.5q^8858.576923076922
-0.5q^11591.576923076922
-0.5q^12306.576923076922
0.5q^13182.576923076922
-0.5q^13223.576923076922
-0.5q^13581.576923076922
-0.5q^17326.576923076922
0.5q^17373.576923076922
0.5q^17783.576923076922
-0.5q^18411.576923076922
-0.5q^18833.576923076922
0.5q^18882.576923076922
0.5q^23257.576923076922
0.5q^23731.576923076922
-0.5q^23786.576923076922
0.5q^24998.576923076922
0.5q^26043.576923076922


In [6]:
z_hat_list_fromvec == z_hat_list

True